# 💧 Module 2: การวิเคราะห์ข้อมูลอุทกวิทยาด้วย Google Earth Engine
## Hydrological Data Analysis with GEE Python API

**Workshop: การใช้ GEE วิเคราะห์ข้อมูลอุตุ-อุทก-ไฟป่า-คุณภาพน้ำ**

---

### 📋 เนื้อหาในบทนี้
1. JRC Global Surface Water - การเปลี่ยนแปลงของพื้นผิวน้ำ
2. Sentinel-1 SAR - การตรวจจับน้ำท่วม
3. MODIS Water Mask
4. การวิเคราะห์พื้นที่น้ำท่วมซ้ำซาก
5. Soil Moisture จาก ERA5-Land
6. การสร้างแผนที่ความเสี่ยงน้ำท่วม

### 📌 พื้นที่ศึกษา: ลุ่มน้ำเจ้าพระยา / ภาคกลางของไทย
### ⏱️ เวลา: ~45 นาที (9:45 - 10:30)

---
## 1. Setup & เชื่อมต่อ GEE

In [ ]:
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# ee.Authenticate()
ee.Initialize(project='YOUR-PROJECT-ID')  # <-- ใส่ GEE Cloud Project ID

plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['figure.dpi'] = 100
print('GEE เชื่อมต่อสำเร็จ ✅')

In [ ]:
# พื้นที่ศึกษา: ภาคกลางของไทย (ลุ่มน้ำเจ้าพระยาตอนล่าง)
thailand = ee.FeatureCollection('FAO/GAUL/2015/level0') \
    .filter(ee.Filter.eq('ADM0_NAME', 'Thailand'))

provinces = ee.FeatureCollection('FAO/GAUL/2015/level1') \
    .filter(ee.Filter.eq('ADM0_NAME', 'Thailand'))

# พื้นที่ภาคกลาง (กรุงเทพและปริมณฑล + จังหวัดใกล้เคียง)
central_provinces = ['Bangkok Metropolis', 'Nonthaburi', 'Pathum Thani',
                     'Ayutthaya', 'Ang Thong', 'Sing Buri', 'Nakhon Pathom']

central_region = provinces.filter(
    ee.Filter.inList('ADM1_NAME', central_provinces)
)

# แสดงพื้นที่ศึกษา
Map = geemap.Map(center=[14.5, 100.5], zoom=8)
Map.addLayer(central_region, {'color': 'blue'}, 'Central Region')
Map

---
## 2. JRC Global Surface Water

**JRC Global Surface Water Mapping**  
- ข้อมูลจาก Landsat (1984-2021)  
- แสดงการเปลี่ยนแปลงของพื้นที่น้ำผิวดิน  
- Dataset: `JRC/GSW1_4/GlobalSurfaceWater`  
- Bands สำคัญ: `occurrence`, `change_abs`, `seasonality`, `recurrence`

In [ ]:
# โหลดข้อมูล JRC Global Surface Water
gsw = ee.Image('JRC/GSW1_4/GlobalSurfaceWater')

# Water Occurrence: ความถี่ที่พบน้ำ (0-100%)
occurrence = gsw.select('occurrence').clip(central_region)

# Seasonality: จำนวนเดือนที่มีน้ำ
seasonality = gsw.select('seasonality').clip(central_region)

# Recurrence: ความถี่ของน้ำที่กลับมาซ้ำ
recurrence = gsw.select('recurrence').clip(central_region)

vis_occurrence = {
    'min': 0,
    'max': 100,
    'palette': ['white', 'lightblue', 'blue', 'darkblue']
}

vis_seasonality = {
    'min': 0,
    'max': 12,
    'palette': ['white', 'cyan', 'blue', 'navy']
}

Map2 = geemap.Map(center=[14.5, 100.5], zoom=9)
Map2.addLayer(occurrence, vis_occurrence, 'Water Occurrence (%)')
Map2.addLayer(seasonality, vis_seasonality, 'Water Seasonality (months)', shown=False)
Map2.addLayer(central_region, {'color': 'red'}, 'Central Region Boundary')
Map2.add_colorbar(vis_occurrence, label='Occurrence (%)', layer_name='Water Occurrence (%)')
Map2

---
## 3. JRC Monthly Water History

ดูพื้นที่น้ำรายเดือน - สามารถเปรียบเทียบช่วงน้ำท่วมกับช่วงปกติ  
Dataset: `JRC/GSW1_4/MonthlyHistory`

In [ ]:
# โหลด JRC Monthly Water History
jrc_monthly = ee.ImageCollection('JRC/GSW1_4/MonthlyHistory') \
    .filterBounds(central_region)

# เปรียบเทียบ: หน้าแล้ง (มี.ค.) vs หน้าฝน (ต.ค.) ปี 2021 (น้ำท่วมใหญ่)
dry_season = jrc_monthly.filterDate('2021-03-01', '2021-03-31').first() \
    .select('water').clip(central_region)
wet_season = jrc_monthly.filterDate('2021-10-01', '2021-10-31').first() \
    .select('water').clip(central_region)

# water band: 0=no data, 1=not water, 2=water
water_vis = {'min': 0, 'max': 2, 'palette': ['white', 'lightgray', 'blue']}

Map3 = geemap.Map(center=[14.5, 100.5], zoom=9)
Map3.addLayer(dry_season, water_vis, 'Dry Season (Mar 2021)')
Map3.addLayer(wet_season, water_vis, 'Wet Season (Oct 2021)')
Map3

In [ ]:
# คำนวณพื้นที่น้ำท่วมเพิ่มเติม (ต่างจากหน้าแล้ง)
# น้ำในหน้าฝนที่ไม่มีในหน้าแล้ง = พื้นที่น้ำท่วม
dry_water = dry_season.eq(2)
wet_water = wet_season.eq(2)
flood_extent = wet_water.subtract(dry_water).selfMask()

Map4 = geemap.Map(center=[14.5, 100.5], zoom=9)
Map4.addLayer(dry_water.selfMask(), {'palette': ['blue']}, 'Permanent Water')
Map4.addLayer(flood_extent, {'palette': ['red']}, 'Flood Extent (Oct 2021)')
Map4

---
## 4. Sentinel-1 SAR สำหรับตรวจจับน้ำท่วม

**Sentinel-1 SAR GRD**  
- ถ่ายภาพได้ทุกสภาพอากาศ (ไม่กลัวเมฆ!)  
- VH polarization เหมาะกับตรวจจับน้ำท่วม  
- น้ำ → ค่า backscatter ต่ำ (สีเข้ม)  
- Dataset: `COPERNICUS/S1_GRD`

In [ ]:
# โหลด Sentinel-1 ช่วงก่อนและหลังน้ำท่วม
s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(central_region) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .select('VH')

# ก่อนน้ำท่วม (ม.ค. 2024 - หน้าแล้ง)
before_flood = s1_collection \
    .filterDate('2024-01-01', '2024-02-28') \
    .mean() \
    .clip(central_region)

# ช่วงน้ำท่วม (ก.ย.-ต.ค. 2024 - หน้าฝน)
during_flood = s1_collection \
    .filterDate('2024-09-01', '2024-10-31') \
    .mean() \
    .clip(central_region)

sar_vis = {'min': -25, 'max': -5}

Map5 = geemap.Map(center=[14.5, 100.5], zoom=9)
Map5.addLayer(before_flood, sar_vis, 'Before (Jan-Feb 2024)')
Map5.addLayer(during_flood, sar_vis, 'During Flood (Sep-Oct 2024)')
Map5

In [ ]:
# ตรวจจับพื้นที่น้ำท่วมจาก SAR
# วิธี: ค่า VH < threshold = น้ำ
threshold = -18  # dB (ปรับได้ตามพื้นที่)

flood_before = before_flood.lt(threshold)
flood_during = during_flood.lt(threshold)

# พื้นที่น้ำท่วมใหม่ = ท่วมตอนหน้าฝน แต่ไม่ท่วมตอนหน้าแล้ง
new_flood = flood_during.subtract(flood_before).gt(0).selfMask()

Map6 = geemap.Map(center=[14.5, 100.5], zoom=9)
Map6.addLayer(during_flood, sar_vis, 'SAR Image')
Map6.addLayer(new_flood, {'palette': ['red']}, 'Detected Flood Area')
Map6.addLayer(central_region, {'color': 'yellow'}, 'Boundary', opacity=0.5)
Map6

In [ ]:
# คำนวณพื้นที่น้ำท่วม (ตร.กม.)
flood_area = new_flood.multiply(ee.Image.pixelArea()).divide(1e6)  # ตร.กม.

total_flood_area = flood_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=central_region.geometry(),
    scale=30,
    maxPixels=1e10
).getInfo()

print(f"พื้นที่น้ำท่วมโดยประมาณ: {total_flood_area['VH']:.2f} ตร.กม.")

---
## 5. Soil Moisture จาก ERA5-Land

**ERA5-Land Soil Moisture**  
- ชั้นดิน 4 ระดับ: 0-7cm, 7-28cm, 28-100cm, 100-289cm  
- Band: `volumetric_soil_water_layer_1` (ชั้นบนสุด)  
- หน่วย: m³/m³

In [ ]:
# โหลด Soil Moisture จาก ERA5-Land Monthly
soil_moisture = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR') \
    .filterDate('2024-01-01', '2024-12-31') \
    .filterBounds(thailand) \
    .select('volumetric_soil_water_layer_1')

# Soil Moisture เดือนตุลาคม
sm_oct = soil_moisture.filter(
    ee.Filter.calendarRange(10, 10, 'month')
).first().clip(thailand)

vis_sm = {
    'min': 0.1,
    'max': 0.5,
    'palette': ['brown', 'yellow', 'green', 'cyan', 'blue']
}

Map7 = geemap.Map(center=[15.87, 100.99], zoom=6)
Map7.addLayer(sm_oct, vis_sm, 'Soil Moisture Oct 2024 (m³/m³)')
Map7.add_colorbar(vis_sm, label='Soil Moisture (m³/m³)',
                  layer_name='Soil Moisture Oct 2024 (m³/m³)')
Map7

In [ ]:
# กราฟ Time Series ความชื้นดินรายเดือน
month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

def get_sm_mean(image):
    stats = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=central_region.geometry(),
        scale=11000,
        maxPixels=1e9
    )
    return image.set('sm_mean', stats.get('volumetric_soil_water_layer_1'))

sm_stats = soil_moisture.map(get_sm_mean)
sm_values = sm_stats.aggregate_array('sm_mean').getInfo()

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(month_labels, sm_values, 'go-', linewidth=2, markersize=8)
ax.fill_between(range(len(sm_values)), sm_values, alpha=0.2, color='green')
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Soil Moisture (m³/m³)', fontsize=12)
ax.set_title('Monthly Soil Moisture - Central Thailand 2024 (ERA5-Land)',
             fontsize=14, fontweight='bold')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

---
## 6. NDWI - ดัชนีน้ำจาก Sentinel-2

**NDWI** (Normalized Difference Water Index)  
$$NDWI = \frac{Green - NIR}{Green + NIR}$$

- NDWI > 0.3 → พื้นที่น้ำ  
- ใช้ Sentinel-2 Surface Reflectance

In [ ]:
# คำนวณ NDWI จาก Sentinel-2
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(central_region) \
    .filterDate('2024-09-01', '2024-10-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
    .median()

ndwi = s2.normalizedDifference(['B3', 'B8']).rename('NDWI').clip(central_region)

# จำแนกพื้นที่น้ำ
water_mask = ndwi.gt(0.3).selfMask()

vis_ndwi = {'min': -0.5, 'max': 0.5, 'palette': ['brown', 'white', 'blue']}

Map8 = geemap.Map(center=[14.5, 100.5], zoom=9)
Map8.addLayer(ndwi, vis_ndwi, 'NDWI')
Map8.addLayer(water_mask, {'palette': ['cyan']}, 'Water Mask (NDWI > 0.3)')
Map8.add_colorbar(vis_ndwi, label='NDWI', layer_name='NDWI')
Map8

---
## 7. Export ผลลัพธ์

In [ ]:
# Export แผนที่น้ำท่วม
task = ee.batch.Export.image.toDrive(
    image=new_flood.unmask(0).toFloat(),
    description='Flood_Detection_Central_Thailand_2024',
    folder='GEE_Workshop',
    fileNamePrefix='flood_central_2024',
    region=central_region.geometry(),
    scale=30,
    crs='EPSG:4326',
    maxPixels=1e10
)
task.start()
print('เริ่ม Export Flood Map ✅')

---
## 📝 สรุป Module 2

| หัวข้อ | Dataset | วิธีการ | ความละเอียด |
|--------|---------|---------|------------|
| พื้นผิวน้ำ | JRC Global Surface Water | Occurrence/Seasonality | 30 m |
| น้ำท่วม (SAR) | Sentinel-1 GRD | Threshold VH < -18 dB | 10 m |
| น้ำท่วม (Optical) | Sentinel-2 SR | NDWI > 0.3 | 10 m |
| ความชื้นดิน | ERA5-Land | Direct reading | ~11 km |

### 💡 Tips
- SAR (Sentinel-1) เหมาะกับช่วงฝนตกเพราะทะลุเมฆได้
- Optical (Sentinel-2) ให้ NDWI ที่แม่นยำกว่าแต่ต้องรอเมฆหาย
- JRC เป็นข้อมูล historical ดีมากสำหรับวิเคราะห์แนวโน้ม
- รวมหลายวิธีเข้าด้วยกัน (Multi-sensor) ได้ผลดีที่สุด

### ☕ พักเบรค 15 นาที (10:30 - 10:45)

---
**➡️ ต่อไป: Module 3 - ไฟป่า (Wildfire)**